## Code from Microsoft Site 
#### https://docs.microsoft.com/en-us/microsoft-365/enterprise/microsoft-365-ip-web-service?view=o365-worldwide
#### IP Addresses are Sorted Correctly

In [1]:
import json
from pathlib import Path
import urllib.request
import uuid
import ipaddress


In [2]:
# helper to call the webservice and parse the response
def webApiGet(methodName, instanceName, clientRequestId):
    ws = "https://endpoints.office.com"
    requestPath = f"{ws}/{methodName}/{instanceName}?clientRequestId={clientRequestId}"
    request = urllib.request.Request(requestPath)
    with urllib.request.urlopen(request) as response:
        return json.loads(response.read().decode())


In [3]:
datapath = Path('2.1 Microsoft URLs and IPs  Exchange Online exchange.txt')
# fetch client ID and version if data exists; otherwise create new file
if datapath.exists():
    with open(datapath, 'r') as fin:
        clientRequestId = fin.readline().strip()
        latestVersion = fin.readline().strip()
else:
    clientRequestId = str(uuid.uuid4())
    latestVersion = '0000000000'
    with open(datapath, 'w') as fout:
        fout.write(clientRequestId + '\n' + latestVersion)


In [4]:
# call version method to check the latest version, and pull new data if version number is different

version = webApiGet('version', 'Worldwide', clientRequestId)

if version['latest'] > latestVersion:
    print('\nNew version of Office 365 worldwide commercial service instance endpoints detected')

    # write the new version number to the data file
    with open(datapath, 'w') as fout:
        fout.write(clientRequestId + '\n' + version['latest'])

    # invoke endpoints method to get the new data
    endpointSets = webApiGet('endpoints', 'Worldwide', clientRequestId)

    flatIps = []
    for endpointSet in endpointSets:
        if endpointSet['category'] in ('Optimize', 'Allow') and endpointSet['serviceArea'] =='Exchange':
            ips = endpointSet['ips'] if 'ips' in endpointSet else []
            ip4s = [ip for ip in ips if '.' in ip]
            flatIps.extend([ip for ip in ip4s])

    print('\nIPv4 Firewall IP Address Ranges for Excahange Online')
    print(', '.join(sorted(set(flatIps), key=ipaddress.IPv4Network)))                        # IP Address Sort

    with open('2.2 Microsoft IPs Exchange Online.txt', 'w') as output_file:
        output_file.write('\n'.join(sorted(set(flatIps), key=ipaddress.IPv4Network)))       # IP Address Seort

    flatUrls = []
    for endpointSet in endpointSets:
        if endpointSet['category'] in ('Optimize', 'Allow') and endpointSet['serviceArea'] =='Exchange':
            urls = endpointSet['urls'] if 'urls' in endpointSet else []
            flatUrls.extend([url for url in urls])

    print('\nURLs for Proxy Server  for Excahange Online')
    print(','.join(sorted(set(flatUrls))))

    with open('2.3 Microsoft URLs Exchange Online.txt', 'w') as output_file:
        output_file.write('\n'.join(sorted(set(flatUrls))))

else:
    print('Office 365 worldwide commercial service instance endpoints are up-to-date')



New version of Office 365 worldwide commercial service instance endpoints detected

IPv4 Firewall IP Address Ranges for Excahange Online
13.107.6.152/31, 13.107.18.10/31, 13.107.128.0/22, 23.103.160.0/20, 40.92.0.0/15, 40.96.0.0/13, 40.104.0.0/15, 40.107.0.0/16, 52.96.0.0/14, 52.100.0.0/14, 52.238.78.88/32, 104.47.0.0/17, 131.253.33.215/32, 132.245.0.0/16, 150.171.32.0/22, 204.79.197.215/32

URLs for Proxy Server  for Excahange Online
*.mail.protection.outlook.com,*.outlook.office.com,*.protection.outlook.com,outlook.office.com,outlook.office365.com,smtp.office365.com
